In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from unidecode import unidecode

In [87]:
def merge_csv_files(base_name, start_year, end_year, folder_path):
    dataframes = []

    for year in range(start_year, end_year + 1):
        filename = f'{base_name}{year}.csv'
        file_path = os.path.join(folder_path, filename)

        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            dataframes.append(df)
        else:
            print(f"Advertencia: El archivo {file_path} no existe y será omitido.")

    if dataframes:
        merged_df = pd.concat(dataframes, ignore_index=True)
    else:
        merged_df = pd.DataFrame()
        print("Advertencia: No se encontraron archivos para fusionar. Se devuelve un DataFrame vacío.")

    return merged_df

In [88]:
def normalizar_columna(df, columna, a_minusculas=True):

    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame")
    
 
    df_normalizado = df.copy()
    
    # Aplicar normalización
    df_normalizado[columna] = df_normalizado[columna].astype(str).apply(
        lambda x: unidecode(x)
            .replace('ñ', 'n').replace('Ñ', 'N') 
            .strip()  
    )

    if a_minusculas:
        df_normalizado[columna] = df_normalizado[columna].str.lower()
    
    return df_normalizado

In [89]:
import pandas as pd
from unidecode import unidecode

def filtrar_municipios_jalisco(df, columna_municipio, verbose=True):
    municipios_oficiales = [
        "Acatic", "Acatlán de Juárez", "Ahualulco de Mercado", "Amacueca", "Amatitán", "Ameca",
        "Arandas", "Atemajac de Brizuela", "Atengo", "Atenguillo", "Atotonilco el Alto",
        "Atoyac", "Autlán de Navarro", "Ayotlán", "Ayutla", "Bolaños", "Cabo Corrientes",
        "Cañadas de Obregón", "Casimiro Castillo", "Chapala", "Chimaltitán", "Chiquilistlán",
        "Cihuatlán", "Cocula", "Colotlán", "Concepción de Buenos Aires", "Cuautitlán de García Barragán",
        "Cuautla", "Cuquío", "Degollado", "Ejutla", "El Arenal", "El Grullo", "El Limón",
        "El Salto", "Encarnación de Díaz", "Etzatlán", "Gómez Farías", "Guachinango", "Guadalajara",
        "Hostotipaquillo", "Huejúcar", "Huejuquilla el Alto", "Ixtlahuacán de los Membrillos",
        "Ixtlahuacán del Río", "Jalostotitlán", "Jamay", "Jesús María", "Jilotlán de los Dolores",
        "Jocotepec", "Juanacatlán", "Juchitlán", "La Barca", "La Huerta", "La Manzanilla de la Paz",
        "Lagos de Moreno", "Magdalena", "Mascota", "Mazamitla", "Mexticacán", "Mezquitic",
        "Mixtlán", "Ocotlán", "Ojuelos de Jalisco", "Pihuamo", "Poncitlán", "Puerto Vallarta",
        "Quitupan", "San Cristóbal de la Barranca", "San Diego de Alejandría", "San Gabriel",
        "San Ignacio Cerro Gordo", "San Juan de los Lagos", "San Juanito de Escobedo",
        "San Julián", "San Marcos", "San Martín de Bolaños", "San Martín Hidalgo",
        "San Miguel el Alto", "San Pedro Tlaquepaque", "San Sebastián del Oeste", "Santa María de los Ángeles",
        "Santa María del Oro", "Sayula", "Tala", "Talpa de Allende", "Tamazula de Gordiano",
        "Tapalpa", "Tecalitlán", "Techaluta de Montenegro", "Tecolotlán", "Tenamaxtlán",
        "Teocaltiche", "Teocuitatlán de Corona", "Tepatitlán de Morelos", "Tequila", "Teuchitlán",
        "Tizapán el Alto", "Tlajomulco de Zúñiga", "Tlaquepaque", "Tolimán", "Tomatlán",
        "Tonalá", "Tonaya", "Tonila", "Totatiche", "Tototlán", "Tuxcacuesco", "Tuxcueca",
        "Tuxpan", "Unión de San Antonio", "Unión de Tula", "Valle de Guadalupe", "Valle de Juárez",
        "Villa Corona", "Villa Guerrero", "Villa Hidalgo", "Villa Purificación", "Yahualica de González Gallo",
        "Zacoalco de Torres", "Zapotiltic", "Zapotitlán de Vadillo", "Zapotlán el Grande", "Zapotlán del Rey",
        "Zapotlanejo"
    ]
    
    # Normalizamos ambas partes para comparación insensible a acentos/mayúsculas/ñ
    municipios_normalizados = {
        unidecode(mun).lower().replace('ñ', 'n'): mun 
        for mun in municipios_oficiales
    }
    
    # Preparamos registro de eliminaciones
    reporte = {
        'eliminados': [],
        'conservados': [],
        'total_original': len(df),
        'total_filtrado': 0
    }
    
    def normalizar_nombre(nombre):
        return unidecode(str(nombre)).lower().replace('ñ', 'n')
    
    # Filtrado
    datos_filtrados = []
    for idx, row in df.iterrows():
        original = row[columna_municipio]
        normalizado = normalizar_nombre(original)
        
        if normalizado in municipios_normalizados:
            # Conservar el nombre oficial (opcional)
            row[columna_municipio] = municipios_normalizados[normalizado]
            datos_filtrados.append(row)
            reporte['conservados'].append(original)
        else:
            reporte['eliminados'].append({
                'indice': idx,
                'valor_original': original,
                'razon': 'No es municipio de Jalisco' if normalizado else 'Valor nulo/vacío'
            })
    
    df_filtrado = pd.DataFrame(datos_filtrados).reset_index(drop=True)
    reporte['total_filtrado'] = len(df_filtrado)
    
    # Reporte detallado
    if verbose:
        print(f"\n=== REPORTE DE FILTRADO ===")
        print(f"• Filas originales: {reporte['total_original']}")
        print(f"• Filas conservadas: {reporte['total_filtrado']}")
        print(f"• Filas eliminadas: {reporte['total_original'] - reporte['total_filtrado']}")
        
        if reporte['eliminados']:
            print("\n🔴 Registros eliminados:")
            for elim in reporte['eliminados'][:10]:  # Muestra primeros 10 para no saturar
                print(f"- Índice {elim['indice']}: '{elim['valor_original']}' ({elim['razon']})")
            
            if len(reporte['eliminados']) > 10:
                print(f"... y {len(reporte['eliminados']) - 10} más")
        
        print("\n✅ Municipios conservados (primeros 5):", reporte['conservados'][:5])
    
    return df_filtrado, reporte if not verbose else df_filtrado

In [90]:
regiones_jalisco = {
    "I Norte": [
        "Bolanos",
        "Chimalitan",
        "Colotan",
        "Huejicar",
        "Huejuquila el Alto",
        "Mezquito",
        "San Martin de Bolanos",
        "Totaiche",
        "Vila Guerrero"
    ],
    "II Altos Norte": [
        "Encarnacion de Diaz",
        "Lagos de Moreno",
        "Ojuelos de Jalisco",
        "San Diego de Alejandria",
        "San Juan de los Lagos",
        "Teocaltiche",
        "Union de San Antonio",
        "Vila Hidalgo"
    ],
    "III Altos Sur": [
        "Acatic",
        "Arandas",
        "Canadas de Obregon",
        "Jalostotitlan",
        "Jesus Maria",
        "Mexticacan",
        "San Ignacio Cerro Gordo",
        "San Julian",
        "San Miguel el Alto",
        "Tepatitlan de Morelos",
        "Valle de Guadalupe",
        "Yahualica de Gonzalez Gallo"
    ],
    "IV Cienega": [
        "Atotonilco el Alto",
        "Ayotlan",
        "Chapala",
        "Degollado",
        "Jamay",
        "Jocotepec",
        "La Barca",
        "Ocotlan",
        "Poncitlan",
        "Tizapan el Alto",
        "Tototlan",
        "Tuxcueca",
        "Zapotlan del Rey"
    ],
    "V Sureste": [
        "Concepcion de Buenos Aires",
        "La Manzanilla de la Paz",
        "Mazamitla",
        "Pihuamo",
        "Quitupan",
        "Santa Maria del Oro",
        "Tamazula de Gordiano",
        "Tecalitlan",
        "Valle de Juarez"
    ],
    "VI Sur": [
        "Amacueca",
        "Atemajac de Brizuela",
        "Atoyac",
        "Gomez Farias",
        "San Gabriel",
        "Sayula",
        "Tapalpa",
        "Techaluta",
        "Tuxpan",
        "Zacoalco de Torres",
        "Zapotiltic",
        "Zapotlan del Vadillo",
        "Zapotlan el Grande"
    ],
    "VII Sierra de Amula": [
        "Atengo",
        "Chiquilistlan",
        "El Grullo",
        "El Limon",
        "Juchitlan",
        "Tecolotlan",
        "Tenamaxtlan",
        "Tonaya",
        "Union de Tula"
    ],
    "VIII Costa Sur": [
        "Autlan de Navarro",
        "Casimiro Castillo",
        "Cihuatlan",
        "La Huerta",
        "Villa Purificacion"
    ],
    "IX Costa Norte": [
        "Cabo Corrientes",
        "Puerto Vallarta",
        "Tomatlan"
    ],
    "X Sierra Occidental": [
        "Atenguillo",
        "Ayutla",
        "Guachinango",
        "Mascota",
        "San Sebastian del Oeste",
        "Talpa de Allende"
    ],
    "XI Valles": [
        "Ahualulco de Mercado",
        "Amatitan",
        "Ameca",
        "Cocula",
        "El Arenal",
        "Etzatlan",
        "Hostotipaquillo",
        "Magdalena",
        "San Juanito de Escobedo",
        "San Marcos",
        "San Martin Hidalgo",
        "Tala",
        "Tequila",
        "Teuchitlan"
    ],
    "XII Centro": [
        "Cuquio",
        "El Salto",
        "Guadalajara",
        "Ixtlahuacan de los Membrillos",
        "Ixtlahuacan del Rio",
        "San Cristobal de la Barranca",
        "San Pedro Tlaquepaque",
        "Tlajomulco de Zuñiga",
        "Tonala",
        "Villa Corona",
        "Zapopan",
        "Zapotlanejo"
    ]
}

# Función para mostrar todas las regiones y sus municipios
def mostrar_regiones():
    for region, municipios in regiones_jalisco.items():
        print(f"\n{region}:")
        for municipio in municipios:
            print(f"- {municipio}")

if __name__ == "__main__":
    print("Regiones y Municipios de Jalisco")
    print("=" * 30)
    mostrar_regiones()

Regiones y Municipios de Jalisco

I Norte:
- Bolanos
- Chimalitan
- Colotan
- Huejicar
- Huejuquila el Alto
- Mezquito
- San Martin de Bolanos
- Totaiche
- Vila Guerrero

II Altos Norte:
- Encarnacion de Diaz
- Lagos de Moreno
- Ojuelos de Jalisco
- San Diego de Alejandria
- San Juan de los Lagos
- Teocaltiche
- Union de San Antonio
- Vila Hidalgo

III Altos Sur:
- Acatic
- Arandas
- Canadas de Obregon
- Jalostotitlan
- Jesus Maria
- Mexticacan
- San Ignacio Cerro Gordo
- San Julian
- San Miguel el Alto
- Tepatitlan de Morelos
- Valle de Guadalupe
- Yahualica de Gonzalez Gallo

IV Cienega:
- Atotonilco el Alto
- Ayotlan
- Chapala
- Degollado
- Jamay
- Jocotepec
- La Barca
- Ocotlan
- Poncitlan
- Tizapan el Alto
- Tototlan
- Tuxcueca
- Zapotlan del Rey

V Sureste:
- Concepcion de Buenos Aires
- La Manzanilla de la Paz
- Mazamitla
- Pihuamo
- Quitupan
- Santa Maria del Oro
- Tamazula de Gordiano
- Tecalitlan
- Valle de Juarez

VI Sur:
- Amacueca
- Atemajac de Brizuela
- Atoyac
- Gomez Fa

In [91]:
municipio_a_region = {}
for region, municipios in regiones_jalisco.items():
    for municipio in municipios:
        nombre_normalizado = unidecode(municipio).lower().replace('ñ', 'n').strip()
        municipio_a_region[nombre_normalizado] = region
municipio_a_region

{'bolanos': 'I Norte',
 'chimalitan': 'I Norte',
 'colotan': 'I Norte',
 'huejicar': 'I Norte',
 'huejuquila el alto': 'I Norte',
 'mezquito': 'I Norte',
 'san martin de bolanos': 'I Norte',
 'totaiche': 'I Norte',
 'vila guerrero': 'I Norte',
 'encarnacion de diaz': 'II Altos Norte',
 'lagos de moreno': 'II Altos Norte',
 'ojuelos de jalisco': 'II Altos Norte',
 'san diego de alejandria': 'II Altos Norte',
 'san juan de los lagos': 'II Altos Norte',
 'teocaltiche': 'II Altos Norte',
 'union de san antonio': 'II Altos Norte',
 'vila hidalgo': 'II Altos Norte',
 'acatic': 'III Altos Sur',
 'arandas': 'III Altos Sur',
 'canadas de obregon': 'III Altos Sur',
 'jalostotitlan': 'III Altos Sur',
 'jesus maria': 'III Altos Sur',
 'mexticacan': 'III Altos Sur',
 'san ignacio cerro gordo': 'III Altos Sur',
 'san julian': 'III Altos Sur',
 'san miguel el alto': 'III Altos Sur',
 'tepatitlan de morelos': 'III Altos Sur',
 'valle de guadalupe': 'III Altos Sur',
 'yahualica de gonzalez gallo': 'III

In [92]:
folder_path = './data/Cuartos_Hoteles'  

merged_data = merge_csv_files('Cuartos_Hoteles_', 2013, 2020, folder_path)

merged_data

,Unnamed: 0,municipio,total,5 estrellas,4 estrellas,3 estrellas,2 estrellas,1 estrellas,sin categoria,año
0,11,Estado,65677,11769,13227,9866,3771,5987,21057,2013
1,12,I Norte,503,0,83,60,9,140,211,2013
2,13,Bolaños,28,0,0,0,0,8,20,2013
3,14,Chimaltitán,8,0,0,0,0,0,8,2013
4,15,Colotlán,138,0,62,0,0,76,0,2013
...,...,...,...,...,...,...,...,...,...,...
936,120,ZAPOTILTIC,54,0,18,0,0,0,36,2020
937,121,ZAPOTITLAN DE VADILLO,20,0,0,0,0,0,20,2020
938,122,ZAPOTLAN DEL REY,28,13,0,15,0,0,0,2020
939,123,ZAPOTLAN EL GRANDE,537,0,137,239,60,101,0,2020


In [93]:
df_normalizado = normalizar_columna(merged_data, 'municipio')
df_normalizado

,Unnamed: 0,municipio,total,5 estrellas,4 estrellas,3 estrellas,2 estrellas,1 estrellas,sin categoria,año
0,11,estado,65677,11769,13227,9866,3771,5987,21057,2013
1,12,i norte,503,0,83,60,9,140,211,2013
2,13,bolanos,28,0,0,0,0,8,20,2013
3,14,chimaltitan,8,0,0,0,0,0,8,2013
4,15,colotlan,138,0,62,0,0,76,0,2013
...,...,...,...,...,...,...,...,...,...,...
936,120,zapotiltic,54,0,18,0,0,0,36,2020
937,121,zapotitlan de vadillo,20,0,0,0,0,0,20,2020
938,122,zapotlan del rey,28,13,0,15,0,0,0,2020
939,123,zapotlan el grande,537,0,137,239,60,101,0,2020


In [94]:
df = filtrar_municipios_jalisco(df_normalizado, 'municipio')



=== REPORTE DE FILTRADO ===
• Filas originales: 941
• Filas conservadas: 913
• Filas eliminadas: 28

🔴 Registros eliminados:
- Índice 0: 'estado' (No es municipio de Jalisco)
- Índice 1: 'i norte' (No es municipio de Jalisco)
- Índice 11: 'ii altos norte' (No es municipio de Jalisco)
- Índice 20: 'iii altos sur' (No es municipio de Jalisco)
- Índice 33: 'iv cienega' (No es municipio de Jalisco)
- Índice 47: 'v sureste' (No es municipio de Jalisco)
- Índice 57: 'vi sur' (No es municipio de Jalisco)
- Índice 71: 'vii sierra de amula' (No es municipio de Jalisco)
- Índice 81: 'viii costa sur' (No es municipio de Jalisco)
- Índice 88: 'ix costa norte' (No es municipio de Jalisco)
... y 18 más

✅ Municipios conservados (primeros 5): ['bolanos', 'chimaltitan', 'colotlan', 'huejucar', 'huejuquilla el alto']
